# <b>Snake Game</b>

In [1]:
import random
import itertools
import csv
import pandas as pd
import pygame
import time
import random
import math



pygame 2.1.3 (SDL 2.0.22, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Training = True
Training = False

learningIteration = 5000
alpha = 0.95
snake_speed = 60

In [3]:
if Training:
    sqs = [''.join(s) for s in list(itertools.product(*[['0', '1']] * 4))]
    widths = ['Left', 'Right', 'Straight']
    heights = ['Up', 'Down', 'Straight']

    states = {}
    for x in widths:
        for y in heights:
            # for k in sqs:
            states[str(x) + "_" + str(y)] = [0, 0, 0, 0]
                # states[str((i, j, k))] = [0, 0, 0, 0]

    # states["Left_UP"]
else:
    import pandas as pd
    df = pd.read_csv("./Trained_Q_Table.csv")
    df
    dic ={}
    # dic = {str(df["Food Location w.r.t snake head"]): df["UP"]}

    key = []
    for i,k in enumerate(df["Food Location w.r.t snake head"]):
        dic.update({str(k): [df["UP"][i], df["DOWN"][i], df["LEFT"][i], df["RIGHT"][i]]})

    states = dic
    # states key value defines the food value w.r.t the snake's head

# Learning Part
#### updating the Q Table

In [4]:
x = 0
y = 1

# Window size
window_x = 720
window_y = 480

# defining colors
black = pygame.Color(0, 0, 0)
white = pygame.Color(255, 255, 255)
red = pygame.Color(255, 0, 0)
green = pygame.Color(0, 255, 0)
blue = pygame.Color(0, 0, 255)

# Initialising pygame
pygame.init()

# Initialise game window
pygame.display.set_caption('Learning GeeksforGeeks Snakes')
game_window = pygame.display.set_mode((window_x, window_y))

# FPS (frames per second) controller
fps = pygame.time.Clock()

# defining snake default position
snake_position = [100, 50]

# defining first 4 blocks of snake body
snake_body = [[100, 50],
			# [90, 50],
			# [80, 50],
			# [70, 50]
			]
# fruit position
fruit_position = [random.randrange(1, (window_x//10)) * 10,
				random.randrange(1, (window_y//10)) * 10]

fruit_spawn = True

# setting default snake direction towards
# right
direction = 'RIGHT'
change_to = direction

# initial score
score = 0

def manhattanD(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def euclidDist(p1, p2):
    return math.dist(p1, p2)

# displaying Score function
def show_score(choice, color, font, size):

	# creating font object score_font
	score_font = pygame.font.SysFont(font, size)
	
	# create the display surface object
	# score_surface
	score_surface = score_font.render('Score : ' + str(score) + " Itr: "+ str(learningIteration), True, color)
	
	# create a rectangular object for the
	# text surface object
	score_rect = score_surface.get_rect()
	
	# displaying text
	game_window.blit(score_surface, score_rect)


def game_over():
    global snake_position, recordMove, Training, score, snake_body
    if Training:
        snake_position = [70, 20]
        # snake_position = [window_x//2, window_y//2]
        
        # snake_position = [random.randrange(1, (window_x//10)) * 10,
        # random.randrange(1, (window_y//10)) * 10]
        # time.sleep(1)
        score = 0
    # game over function
    else:
        # creating font object my_font
        my_font = pygame.font.SysFont('times new roman', 50)

        # creating a text surface on which text
        # will be drawn
        game_over_surface = my_font.render('Your Score is : ' + str(score), True, red)

        # create a rectangular object for the text
        # surface object
        game_over_rect = game_over_surface.get_rect()

        # setting position of the text
        game_over_rect.midtop = (window_x/2, window_y/4)

        # blit will draw the text on screen
        game_window.blit(game_over_surface, game_over_rect)
        pygame.display.flip()

        # after 2 seconds we will quit the
        # program
        time.sleep(2)
        snake_position = [70, 20]
        snake_body = [[70, 50]]
        score = 0
#         # deactivating pygame library
#         pygame.quit()

#         # quit the program
#         quit()

    
    
    

def stateKeyPredict(snake_position, fruit_position):
    statekey = ""
    if snake_position[x] - fruit_position[x] == 0 and snake_position[y] - fruit_position[y] == 0:
        statekey = statekey + "Straight_Straight"
        
    elif snake_position[x] - fruit_position[x] == 0  and snake_position[y] > fruit_position[y]:
        statekey = statekey + "Straight_Down"
        
    elif snake_position[x] - fruit_position[x] == 0  and snake_position[y] < fruit_position[y]:
        statekey = statekey + "Straight_Up"
        
    elif snake_position[x] < fruit_position[x] and snake_position[y] - fruit_position[y] == 0:
        statekey = statekey + "Right_Straight"
        
    elif snake_position[x] < fruit_position[x]  and snake_position[y] < fruit_position[y]:
        statekey = statekey + "Right_Down"
        
    elif snake_position[x] < fruit_position[x] and snake_position[y] > fruit_position[y]:
        statekey = statekey + "Right_Up"

    elif snake_position[x] > fruit_position[x] and snake_position[y] - fruit_position[y] == 0:
        statekey = statekey + "Left_Straight"
        
    elif snake_position[x] > fruit_position[x] and snake_position[y] > fruit_position[y]:
        statekey = statekey + "Left_Down"
        
    elif snake_position[x] > fruit_position[x] and snake_position[y] < fruit_position[y]:
        statekey = statekey + "Left_Up"
    
    else:
        print("in else", statekey)
    return statekey

# Main Function
#directionList = [0,      1,      2,      3]
directionList = ['UP', 'DOWN', 'LEFT', 'RIGHT']
recordMove = []
while learningIteration:
    global recordMove
    if Training:
        learningIteration -= 1
    else:
        learningIteration = 1
    # Current_Distance = manhattanD(snake_position, fruit_position)
    Current_Distance = euclidDist(snake_position, fruit_position)
    Reward = 0
    
    
    # print(snake_position, fruit_position)
    currStateKey = stateKeyPredict(snake_position, fruit_position)
    
    
# checking the Q Table for keypress
    Max_Val_For_Statekey = max(states[currStateKey])
    
    if Max_Val_For_Statekey == 0:
        randX = random.choice([0, 1, 2, 3])
        change_to = directionList[randX]
    else:
        change_to = directionList[states[currStateKey].index(Max_Val_For_Statekey)]
    
    indexNumber = states[currStateKey].index(Max_Val_For_Statekey)
    # print(Max_Val_For_Statekey, statekey, change_to, "\n")
    
    if change_to == 'UP' and direction != 'DOWN':
        direction = 'UP'
    if change_to == 'DOWN' and direction != 'UP':
        direction = 'DOWN'
    if change_to == 'LEFT' and direction != 'RIGHT':
        direction = 'LEFT'
    if change_to == 'RIGHT' and direction != 'LEFT':
        direction = 'RIGHT'

    # Moving the snake
    if direction == 'UP':
        snake_position[1] -= 10
    if direction == 'DOWN':
        snake_position[1] += 10
    if direction == 'LEFT':
        snake_position[0] -= 10
    if direction == 'RIGHT':
        snake_position[0] += 10
    
    # checking the updated distance and updating the reward accordingly
    New_Distance = manhattanD(snake_position, fruit_position)
    
    if New_Distance < Current_Distance:
        Reward = 1
    else:
        Reward = -0.50
    
    # nextStateKey = stateKeyPredict(snake_position, fruit_position)
    nextStateKey = euclidDist(snake_position, fruit_position)
    
    # Snake body growing mechanism
    # if fruits and snakes collide then scores
    # will be incremented by 10
    snake_body.insert(0, list(snake_position))
    if snake_position[0] == fruit_position[0] and snake_position[1] == fruit_position[1]:
        score += 1
        fruit_spawn = False
        if Training:
            snake_body.pop()
    else:
        snake_body.pop()

    if not fruit_spawn:
        fruit_position = [random.randrange(1, ((window_x-60)//10)) * 10,
                        random.randrange(1, ((window_y-0)//10)) * 10]

    fruit_spawn = True
    game_window.fill(black)

    for pos in snake_body:
        pygame.draw.rect(game_window, green,
                        pygame.Rect(pos[0], pos[1], 10, 10))
    pygame.draw.rect(game_window, white, pygame.Rect(
        fruit_position[0], fruit_position[1], 10, 10))

    # Game Over conditions
    if snake_position[0] < 0 or snake_position[0] > window_x-10:
        Reward = -1
        game_over()
    if snake_position[1] < 0 or snake_position[1] > window_y-10:
        Reward = -1
        game_over()

    # Touching the snake body
    for block in snake_body[1:]:
        if snake_position[0] == block[0] and snake_position[1] == block[1]:
            Reward = -1
            game_over()

 
    # updating Q table:
    if Training:
        Qv_T = ((1-alpha)*states[currStateKey].index(Max_Val_For_Statekey)) + (alpha*(Reward + (0.5*max(states[currStateKey]))))

        states[currStateKey][indexNumber] = Qv_T
    # print(states, "\n\n")
    
    # displaying score countinuously
    show_score(1, white, 'times new roman', 40) 

    # Refresh game screen
    pygame.display.update()

    # Frame Per Second /Refresh Rate
    fps.tick(snake_speed)

print(states)

# saving the trained data in trained Q table
if Training:
    import csv
    fields: list[str] = ['Food Location w.r.t snake head', 'UP', 'DOWN', 'LEFT', 'RIGHT']
    filename = "Trained_Q_Table.csv"

    with open('Trained_Q_Table.csv', 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerow(fields)

    with open('Trained_Q_Table(euclidean).csv', 'a') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)
        for k in states:
            write.writerow([k, states[k][0], states[k][1], states[k][2], states[k][3]])
            # write.writerow(states[k])


KeyboardInterrupt

